In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Conv1D , Dropout , Flatten , MaxPooling1D, Dense, Input, BatchNormalization
from keras.layers.core import Lambda
from keras.models import Model , load_model

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
from IPython.display import Image

Using TensorFlow backend.


In [2]:
def ApogeeDR14GaiaDR2(dim_t , dim_n): 
    """
    INPUT: 
    dim_t - number of time steps of spectrum 
    dim_n - number of features of spectrum
    """
    
    #SPECTRUM TO LUINOSITY
    dim_1 = 1 # number of corrected magnitude for one example 
    units = 1 #number of final output for one example+
    
    inputs_spectra = Input(shape=(dim_t, dim_n), name="pseudo-lum-input") 
    inputs_mag = Input(shape=(dim_1,), name="K_mag")
    
    x_parallax = Conv1D(filters=2, kernel_size=3, activation='relu')(inputs_spectra)
    x_parallax = BatchNormalization()(x_parallax)
    x_parallax = MaxPooling1D(pool_size=2)(x_parallax)
    
    x_parallax = Conv1D(filters=4, kernel_size=3, activation='relu')(x_parallax)
    x_parallax = BatchNormalization()(x_parallax)
    x_parallax = MaxPooling1D(pool_size=2)(x_parallax)
    
    x_parallax = Flatten()(x_parallax)
    x_parallax = Dense(128, activation='relu')(x_parallax) 
    x_parallax = Dense(64, activation='relu')(x_parallax) 
    x_parallax = Dense(32, activation='relu')(x_parallax)
    x_parallax = Dense(units, activation='softplus', name="pseudo-lum")(x_parallax) 
      
    #OFFSET CORRECTION : (optimization)
    inputs_offset = Input(shape=(3,), name="offset-input")
    x_offset = Dense(64, activation='relu')(inputs_offset)
    x_offset = Dense(32, activation='relu')(x_offset) 
    x_offset = Dense(units, activation='tanh', name="offset")(x_offset) 
    
    #Functions
    outputs_parallax = Lambda(lambda function: tf.math.multiply(function[0], tf.math.pow(10., 
                              tf.math.multiply(-0.2, function[1]))),
                              name='parallax')([x_parallax, inputs_mag])
    
    outputs_parallax_with_offset = Lambda(lambda function: tf.math.add(function[0], function[1]),
                                          name="sum-parallax-offset")([outputs_parallax, x_offset]) 
    
    #Model setup
    model =  Model(inputs = [inputs_spectra,inputs_mag, inputs_offset],outputs = [outputs_parallax_with_offset])
    
    return model 

#### Data load 

In [3]:
# Cargamos los datos
path_local_data = '/home/anell/Desktop/Bovy/AnellExercises/Fits_files'
#path_local_data = '/home/bapanes/Research-Now/local/astronn-local/apo-gaia/'

with h5py.File(f'{path_local_data}/apogeedr14_gaiadr2_with_spectrum_probando_rendimiento_2.h5','r') as F:  
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kmag = np.array(F['corrected_magnitude_K'])
    bp_rp = np.array(F['bp_rp'])
    Gmag = np.array(F['phot_g_mean_mag'])
    teff = np.array(F['NN_teff'])
    apogee_id = np.array(F['APOGEE_ID'])
    snr = np.array(F['SNR'])
    fe_h = np.array(F['Fe/H'])
    path_spectra = np.array(F['Path_spectra'])

In [4]:
parallax.shape , spectra.shape , Kmag.shape

((60986,), (60986, 7514), (60986,))

In [5]:
#Establecemos las variables que entrarán a la red y corregimos sus dimensiones
X = np.expand_dims(spectra, axis = 2)
Y = np.expand_dims(parallax, axis = 1)
K_mag = np.expand_dims(Kmag, axis = 1)

# Normalizamos Gmag , el color (G_bp - G_rp) y teff
Gmag_std = np.std(Gmag)
Gmag_mean = np.mean(Gmag)
Gmag_norm = (Gmag - Gmag_mean) / Gmag_std

bp_rp_std = np.std(bp_rp)
bp_rp_mean = np.mean(bp_rp)
bp_rp_norm = (bp_rp - bp_rp_mean) / bp_rp_std

teff_std = np.std(teff)
teff_mean = np.mean(teff)
teff_norm = (teff - teff_mean) / teff_std

G_mag = np.expand_dims(Gmag_norm, axis=1)
Bp_Rp = np.expand_dims(bp_rp_norm, axis=1)
Teff = np.expand_dims(teff_norm, axis=1)

X_offset = np.concatenate((G_mag, Bp_Rp , Teff), axis = 1) 

In [6]:
print(X.shape , Y.shape, K_mag.shape, X_offset.shape)

(60986, 7514, 1) (60986, 1) (60986, 1) (60986, 3)


#### SNR cut

In [7]:
high_snr_idx = []
low_snr_idx = []

for i in range(len(snr)):
    if snr[i] >= 200:           
        high_snr_idx.append(i)
    else:
        low_snr_idx.append(i)

random.seed(10)
random.shuffle(high_snr_idx)
random.seed(60)
random.shuffle(low_snr_idx)

#### Estos valores pueden ser comparados con Bovy train-test separation, 

Apart from taht we should be more clear about data preparation and evaluation

get consistent and repeatable accuracy using train and validation approach <br>
undestand the process of validation (draft about probability_distributions) <br> 
check the precision on the test data using baseline separation <br>
compute learning curve, following Fig 1. of Nguyen et. al.<br>
try with percentual mse, such that low and high parallax values weight the same <br>

initially we can consider SNR>200 for train-valid and SNR<200 for test <br>
understand why it is possible to make this separation and hope for consistent results <br>

compare to Bovy using this data separation, using global precision and learning curve<br>

How well are we doing with respect to Bovy?

In [10]:
#Image("/home/bapanes/Research-Now/TesisAnell/Figures/learning_curve_genoma.png")
#Image("learning_curve_genoma.png")

In [9]:
print("SNR>200: %d, else: %d"%(len(high_snr_idx), len(low_snr_idx)))

SNR>200: 27721, else: 33265


### Diseño Experimental:

Consideraré train (0.8) y valid (0.2)

train_val_1 $\rightarrow$  177 (train) + 44 (valid) = 221

train_val_2 $\rightarrow$  400 (train) + 100 (valid) = 500

train_val_3 $\rightarrow$  800 (train) + 200 (valid) = 1000

train_val_4 $\rightarrow$  2400 (train) + 600 (valid) = 3000

train_val_5 $\rightarrow$  6400 (train) + 1600 (valid) = 8000

train_val_6 $\rightarrow$  12000 (train) + 3000 (valid) = 15000

train_val_7 $\rightarrow$  22177 (train) + 5594 (valid) = 27771

test $\rightarrow$ 33265

In [11]:
X_high_snr = X[high_snr_idx]
Y_high_snr = Y[high_snr_idx]
K_mag_high_snr = K_mag[high_snr_idx]
X_offset_high_snr = X_offset[high_snr_idx]

X_low_snr = X[low_snr_idx]
Y_low_snr = Y[low_snr_idx]
K_mag_low_snr = K_mag[low_snr_idx]
X_offset_low_snr = X_offset[low_snr_idx]

In [12]:
X_train_val_1 = X_high_snr[:221]
Y_train_val_1 = Y_high_snr[:221]
K_mag_train_val_1 = K_mag_high_snr[:221]
X_offset_train_val_1 = X_offset_high_snr[:221]

X_train_val_2 = X_high_snr[221:721]
Y_train_val_2 = Y_high_snr[221:721]
K_mag_train_val_2 = K_mag_high_snr[221:721]
X_offset_train_val_2 = X_offset_high_snr[221:721]

X_train_val_3 = X_high_snr[721:1721]
Y_train_val_3 = Y_high_snr[721:1721]
K_mag_train_val_3 = K_mag_high_snr[721:1721]
X_offset_train_val_3 = X_offset_high_snr[721:1721]

X_train_val_4 = X_high_snr[1721:4721]
Y_train_val_4 = Y_high_snr[1721:4721]
K_mag_train_val_4 = K_mag_high_snr[1721:4721]
X_offset_train_val_4 = X_offset_high_snr[1721:4721]

X_train_val_5 = X_high_snr[4721:12721]
Y_train_val_5 = Y_high_snr[4721:12721]
K_mag_train_val_5 = K_mag_high_snr[4721:12721]
X_offset_train_val_5 = X_offset_high_snr[4721:12721]

X_train_val_6 = X_high_snr[12721:]
Y_train_val_6 = Y_high_snr[12721:]
K_mag_train_val_6 = K_mag_high_snr[12721:]
X_offset_train_val_6 = X_offset_high_snr[12721:]

X_train_val_7 = X_high_snr
Y_train_val_7 = Y_high_snr
K_mag_train_val_7 = K_mag_high_snr
X_offset_train_val_7 = X_offset_high_snr

X_test = X_low_snr
Y_test = Y_low_snr
K_mag_test = K_mag_low_snr
X_offset_test = X_offset_low_snr
snr_test = snr[low_snr_idx]

In [13]:
print(Y_train_val_1.shape , Y_train_val_2.shape , Y_train_val_3.shape, Y_train_val_4.shape , 
      Y_train_val_5.shape,Y_train_val_6.shape ,Y_train_val_7.shape , Y_test.shape)

(221, 1) (500, 1) (1000, 1) (3000, 1) (8000, 1) (15000, 1) (27721, 1) (33265, 1)


#### Model

In [14]:
n_timesteps, n_features = X_train_val_1.shape[1], X_train_val_1.shape[2]

Global_model = ApogeeDR14GaiaDR2(n_timesteps , n_features)

Global_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pseudo-lum-input (InputLayer)   (None, 7514, 1)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 7512, 2)      8           pseudo-lum-input[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 7512, 2)      8           conv1d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3756, 2)      0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [15]:
Global_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

path_local = "/home/anell/Desktop/Bovy/AnellExercises/Good_Models/SNR100/"

earlystopper = EarlyStopping(monitor='val_loss', patience=50, verbose=1, min_delta=1e-7)
checkpoint = ModelCheckpoint(f'{path_local}/Modelo_1_D7_ReplicaBovy.h5', monitor='val_loss', 
                             verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, verbose=1, patience=5, min_lr=0.000000001)

callbacks=[reduce_lr, checkpoint, earlystopper]

Global_model.fit([X_train_val_7, K_mag_train_val_7, X_offset_train_val_7], Y_train_val_7, callbacks=callbacks,
                 epochs=200, batch_size=128, verbose=1, shuffle="batch", validation_split=0.2)


Train on 22176 samples, validate on 5545 samples
Epoch 1/200
22176/22176 [==============================] - 19s 847us/step - loss: 6.3059 - mse: 6.3059 - val_loss: 18.5814 - val_mse: 18.5814

Epoch 00001: val_loss improved from inf to 18.58138, saving model to /home/anell/Desktop/Bovy/AnellExercises/Good_Models//Modelo_1_D7_ReplicaBovy.h5
Epoch 2/200
22176/22176 [==============================] - 17s 778us/step - loss: 1.5673 - mse: 1.5673 - val_loss: 14.2141 - val_mse: 14.2141

Epoch 00002: val_loss improved from 18.58138 to 14.21409, saving model to /home/anell/Desktop/Bovy/AnellExercises/Good_Models//Modelo_1_D7_ReplicaBovy.h5
Epoch 3/200
22176/22176 [==============================] - 18s 790us/step - loss: 0.8595 - mse: 0.8595 - val_loss: 6.3522 - val_mse: 6.3522

Epoch 00003: val_loss improved from 14.21409 to 6.35215, saving model to /home/anell/Desktop/Bovy/AnellExercises/Good_Models//Modelo_1_D7_ReplicaBovy.h5
Epoch 4/200
22176/22176 [==============================] - 18s 819us


Epoch 00036: val_loss did not improve from 0.33224
Epoch 37/200
22176/22176 [==============================] - 18s 808us/step - loss: 0.0601 - mse: 0.0601 - val_loss: 0.5184 - val_mse: 0.5184

Epoch 00037: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00037: val_loss did not improve from 0.33224
Epoch 38/200
22176/22176 [==============================] - 18s 806us/step - loss: 0.0505 - mse: 0.0505 - val_loss: 0.3848 - val_mse: 0.3848

Epoch 00038: val_loss did not improve from 0.33224
Epoch 39/200
22176/22176 [==============================] - 18s 804us/step - loss: 0.0490 - mse: 0.0490 - val_loss: 0.3651 - val_mse: 0.3651

Epoch 00039: val_loss did not improve from 0.33224
Epoch 40/200
22176/22176 [==============================] - 18s 805us/step - loss: 0.0484 - mse: 0.0484 - val_loss: 0.3592 - val_mse: 0.3592

Epoch 00040: val_loss did not improve from 0.33224
Epoch 41/200
22176/22176 [==============================] - 18s 808us/step - loss: 0.0477 - mse

22176/22176 [==============================] - 18s 833us/step - loss: 0.0382 - mse: 0.0382 - val_loss: 0.3521 - val_mse: 0.3521

Epoch 00076: val_loss did not improve from 0.33224
Epoch 77/200
22176/22176 [==============================] - 18s 819us/step - loss: 0.0381 - mse: 0.0381 - val_loss: 0.3505 - val_mse: 0.3505

Epoch 00077: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 00077: val_loss did not improve from 0.33224
Epoch 78/200
22176/22176 [==============================] - 21s 932us/step - loss: 0.0381 - mse: 0.0381 - val_loss: 0.3522 - val_mse: 0.3522

Epoch 00078: val_loss did not improve from 0.33224
Epoch 79/200
22176/22176 [==============================] - 22s 981us/step - loss: 0.0381 - mse: 0.0381 - val_loss: 0.3525 - val_mse: 0.3525

Epoch 00079: val_loss did not improve from 0.33224
Epoch 80/200
22176/22176 [==============================] - 21s 925us/step - loss: 0.0381 - mse: 0.0381 - val_loss: 0.3518 - val_mse: 0.3518

Epoch 00080: val_l

#### simple evaluations on test sample (SNR < 200)

In [16]:
J_test , mse_test = Global_model.evaluate([X_test, K_mag_test , X_offset_test], Y_test)

33265/33265 [==============================] - 10s 302us/step


In [17]:
print(J_test,mse_test)

350.5836360920646 350.5837097167969


In [18]:
pred = Global_model.predict([X_test, K_mag_test , X_offset_test])

In [19]:
target = Y_test

In [ ]:
plt.scatter(target, pred, c="g", alpha=0.5)

plt.xlabel("target")
plt.ylabel("pred")

plt.ylim(0,100)
plt.xlim(0,100)
plt.show()